# 📌 Telco Churn — Advanced Modeling (Notebook n°4)

Objectif :
- entraîner un modèle plus avancé que la baseline (LogReg)
- comparer les performances (ROC-AUC + Recall churn)
- conserver un pipeline reproductible (preprocess + modèle)
- sauvegarder le meilleur modèle

In [17]:
# Imports

import numpy as np
import pandas as pd
import joblib

from joblib import load, dump

from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support
)

from sklearn.ensemble import RandomForestClassifier


In [5]:
# Load data & split X/y

train_df = pd.read_csv("../../data/processed/telco_train.csv")
test_df  = pd.read_csv("../../data/processed/telco_test.csv")

X_train = train_df.drop(columns=["Churn"])
y_train = train_df["Churn"]

X_test  = test_df.drop(columns=["Churn"])
y_test  = test_df["Churn"]

print("Train:", X_train.shape, "| churn rate:", round(y_train.mean(), 3))
print("Test :", X_test.shape,  "| churn rate:", round(y_test.mean(), 3))


Train: (5634, 21) | churn rate: 0.265
Test : (1409, 21) | churn rate: 0.265


In [20]:
# Save train/test sets for future notebooks

joblib.dump(X_train, "../../data/processed/X_train.joblib")
joblib.dump(X_test, "../../data/processed/X_test.joblib")
joblib.dump(y_train, "../../data/processed/y_train.joblib")
joblib.dump(y_test, "../../data/processed/y_test.joblib")


['../data/processed/y_test.joblib']

In [8]:
# Load preprocessor

from joblib import load

preprocessor = load("../../models/preprocessor.joblib")
preprocessor


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['SeniorCitizen', 'tenure', 'MonthlyCharges',
                                  'TotalCharges', 'customer_value',
                                  'high_monthly_charges']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['gender', 'Partner', 'Dependents',
                                  'PhoneService', 'MultipleLines',
                                  'InternetService', 'OnlineSecurity',
                                  'OnlineBackup', 'DeviceProtection',
                                  'TechSupport', 'StreamingTV',
                                  'StreamingMovies', 'Contract',
                                  'PaperlessBilling', 'PaymentMethod'])])

In [9]:
# Build pipeline with RandomForest

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        (
            "model",
            RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_split=5,
                min_samples_leaf=2,
                class_weight="balanced",
                random_state=42,
                n_jobs=-1
            )
        )
    ]
)

rf_pipeline


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'customer_value',
                                                   'high_monthly_charges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHo...
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced',
                                        min_samples_leaf=2, min_samples_split=5,
                                        n_estimators=300, n_jobs=-1,
                                        random_state=42))])

In [10]:
# Train RandomForest model

rf_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'customer_value',
                                                   'high_monthly_charges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHo...
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('model',
                 RandomForestClassifier(class_weight='balanced',
                                        min_samples_leaf=2, min_samples_split=5,
                                        n_estimators=300, n_jobs=-1,
                                        random_state=42))])

In [11]:
# Evaluate on test set

from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix
)

# Prédictions
y_pred_rf = rf_pipeline.predict(X_test)

# Probabilités churn
y_proba_rf = rf_pipeline.predict_proba(X_test)[:, 1]

print("ROC-AUC RandomForest :", round(roc_auc_score(y_test, y_proba_rf), 3))


ROC-AUC RandomForest : 0.835


In [12]:
# Classification report

print("Classification report — RandomForest\n")
print(classification_report(y_test, y_pred_rf))


Classification report — RandomForest

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1035
           1       0.58      0.65      0.61       374

    accuracy                           0.78      1409
   macro avg       0.73      0.74      0.73      1409
weighted avg       0.79      0.78      0.79      1409



In [13]:
# Confusion matrix

confusion_matrix(y_test, y_pred_rf)


array([[862, 173],
       [132, 242]], dtype=int64)

In [14]:
# Model comparison

results = pd.DataFrame({
    "model": ["Logistic Regression", "Random Forest"],
    "roc_auc": [
        0.841,  # valeur NB3
        roc_auc_score(y_test, y_proba_rf)
    ],
    "recall_churn": [
        0.786,  # recall à threshold 0.5 pour LogReg
        classification_report(y_test, y_pred_rf, output_dict=True)["1"]["recall"]
    ]
})

results


,model,roc_auc,recall_churn
0,Logistic Regression,0.841000,0.786000
1,Random Forest,0.835392,0.647059


## Conclusion — Modélisation avancée

Un modèle RandomForest a été entraîné et comparé à la baseline
(Logistic Regression) en conservant le même pipeline de preprocessing.

Résultats :
- Logistic Regression : ROC-AUC = 0.84, Recall churn ≈ 0.79
- RandomForest        : ROC-AUC = 0.83, Recall churn ≈ 0.65

Bien que le RandomForest améliore légèrement la précision,
il détecte significativement moins de clients à risque.

Dans un objectif de rétention client, la Logistic Regression est retenue
comme modèle final en raison de son meilleur compromis
entre performance et impact métier.